In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [ ]:
titanic = pd.read_csv('../input/train.csv')

In [ ]:
titanic[['Age']].describe()

In [ ]:
sns.heatmap(titanic.isnull(),yticklabels=False,cbar=False,cmap='viridis')
plt.show()

In [ ]:
sns.countplot(x='Survived',data=titanic,palette='RdBu_r');

In [ ]:
sns.countplot(x='Survived',hue='Sex',data=titanic,palette='RdBu_r');

In [ ]:
sns.countplot(x='Survived',hue='Pclass',data=titanic,palette='viridis');

In [ ]:
sns.countplot(x='Pclass',hue='Survived',data=titanic,palette='viridis');

In [ ]:
titanic[titanic.Survived == 0].hist("Age", bins=20)
plt.show()

In [ ]:
titanic.Age = titanic.groupby('Pclass').Age.transform(lambda x: x.fillna(np.round(x.mean())))

In [ ]:
# Extract titles from name
titanic['Title']=0
for i in titanic:
    titanic['Title']=titanic['Name'].str.extract('([A-Za-z]+)\.', expand=False)  # Use REGEX to define a search pattern

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x=titanic['Title'], y=titanic['Age']);
plt.show()

In [ ]:
titanic_melted = pd.melt(titanic[['Survived', 'Age']], "Survived", var_name="Ages")

In [ ]:
titanic_melted.head()

In [ ]:
sns.swarmplot(x = 'Ages', y="value", hue="Survived", data=titanic_melted)
plt.show()

In [ ]:
df = titanic.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
sex_conv = lambda x : 1 if x=='male' else 0 
df['Is_male'] = df['Sex'].apply(sex_conv)

In [ ]:
df['Age'] =pd.cut(df['Age'], bins=[1, 12, 50, 200], labels=['Child','Adult','Elder'])

In [ ]:
embarked_new = pd.get_dummies(df['Embarked'])
train = pd.concat([df[['PassengerId','Survived','Pclass','Sex', 
                       'Age', 'SibSp', 'Parch', 'Fare','Embarked','Is_male']], embarked_new], axis=1)

In [ ]:
# Family size feature
df['FamilySize'] = df['SibSp'] + df['Parch']
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)
df.head()

In [ ]:
titanic = pd.read_csv('../input/train.csv')
df = titanic.drop(['Ticket', 'Cabin'], axis=1)

#Converting Sex
sex_conv = lambda x : 1 if x=='male' else 0 
df['Is_male'] = df['Sex'].apply(sex_conv)

# Adding "Title" column
df['Title']=0
for i in df:
    df['Title']=df['Name'].str.extract('([A-Za-z]+)\.', expand=False)  

# Imputing Age  
df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(np.round(x.mean())))

# Converting Age
df['Age'] =pd.cut(df['Age'], bins=[1, 12, 50, 200], labels=['Child','Adult','Elder'])

# Adding Family column
df['FamilySize'] = df['SibSp'] + df['Parch']
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)

#Binilen limanları gruplandırma
embarked_new = pd.get_dummies(df['Embarked'])

train = pd.concat([df[['Survived','Pclass', 
                       'FamilySize','Fare',
                       'Is_male', 'Age']], embarked_new], axis=1)

X = train[['Is_male','Pclass', 'Age', "FamilySize"]]
y = train['Survived']